In [2]:
import pandas as pd

In [3]:
df_menu = pd.read_excel('./menu.xlsx', header=0)
list_url = df_menu['Url'].to_list()

print('总数量：' + str(len(list_url)))
list_url

总数量：4


['https://www.dormanproducts.com/p-10376-645-702.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-24889-85178.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-65606-645-596.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-50806-84645.aspx?origin=keyword']

In [4]:
df = pd.DataFrame(columns=['Url',
                           'OE_Numbers',
                           'vehicle_url',
                           'Src',
                           'status'])

df

,Url,OE_Numbers,vehicle_url,Src,status


In [5]:
import requests

In [6]:
i = 0

resp = requests.get(list_url[i]).text

resp

'\r\n<!doctype html>\r\n<html lang="en">\r\n<head>\r\n    <title>Electrical Pigtail | 645-702 | Pigtail 7-Wire | Dorman Products</title>\r\n\t<meta charset="utf-8" />\r\n    <meta http-equiv="cache-control" content="no-cache, no-store, must-revalidate" />\r\n    <meta http-equiv="pragma" content="no-cache" />\r\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n    <meta http-equiv="expires" content="-1" />\r\n\r\n    <meta name="description" content="Dorman Products - 645-702 : Pigtail 7-Wire. Make better electrical repairs and accessory installations with this pigtail connector. Designed for a clean appearance when installed, it features weatherproof terminals and a locking tab for a reliable quick disconnect electrical connection." />\r\n    <meta name="keywords" content="645-702,  Pigtail, Pigtails, General Use Electrical Connector, Headlight, Control, Module, Connector, Headlights, Controls, Modules, Connectors, Headlight Control Module Electrical Plug, 

In [7]:
from bs4 import BeautifulSoup

In [8]:
soup = BeautifulSoup(resp, 'lxml')

print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   Electrical Pigtail | 645-702 | Pigtail 7-Wire | Dorman Products
  </title>
  <meta charset="utf-8"/>
  <meta content="no-cache, no-store, must-revalidate" http-equiv="cache-control"/>
  <meta content="no-cache" http-equiv="pragma"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="-1" http-equiv="expires"/>
  <meta content="Dorman Products - 645-702 : Pigtail 7-Wire. Make better electrical repairs and accessory installations with this pigtail connector. Designed for a clean appearance when installed, it features weatherproof terminals and a locking tab for a reliable quick disconnect electrical connection." name="description"/>
  <meta content="645-702,  Pigtail, Pigtails, General Use Electrical Connector, Headlight, Control, Module, Connector, Headlights, Controls, Modules, Connectors, Headlight Control Module Electrical Plug, Windshield, Wiper, Switch, Connector, Wipers, Switches, Connectors,

In [9]:
with open('./part.html', 'w', encoding='utf-8') as file:
    file.write(str(soup))

In [10]:
from lxml import etree

In [11]:
html = etree.HTML(str(soup))

oe = ';'.join(html.xpath('//section[@id="productOE"]/descendant::*/th[@scope="row"]/text()'))

oe

'15862656'

In [12]:
if len(html.xpath('//input[@id="hidfDetailApp"]/@value')) == 0:
    vehicle_url = ''
else:
    vehicle_url = 'https://www.dormanproducts.com/' + html.xpath('//input[@id="hidfDetailApp"]/@value')[0]

vehicle_url

''

In [13]:
src = html.xpath('//img[@id="imgProductMain"]/@src')[0]

src

'https://static.dormanproducts.com/images/product/icon/645-702-007.JPG'

In [14]:
df_temp = pd.DataFrame([{'Url': list_url[i],
                         'OE_Numbers': oe,
                         'vehicle_url': vehicle_url,
                         'Src': src,
                         'status': 'ok'}])

df_temp

,Url,OE_Numbers,vehicle_url,Src,status
0,https://www.dormanproducts.com/p-10376-645-702...,15862656,,https://static.dormanproducts.com/images/produ...,ok


In [15]:
list_row = html.xpath('//section[@id="productSpec"]/descendant::tr')
for row in list_row:
    df_temp[row.xpath('./th')[0].text.strip()[:-1].lower()] = row.xpath('./td')[0].text
    
df = pd.concat([df, df_temp], ignore_index=True).fillna('')

df

,Url,OE_Numbers,vehicle_url,Src,status,overall length (in),package contents,package quantity,packaging type,terminal gender,terminal material,terminal type,wire gauge measurement,wire quantity
0,https://www.dormanproducts.com/p-10376-645-702...,15862656,,https://static.dormanproducts.com/images/produ...,ok,19 in,7-wire Harness,1,Box,Female,Metal,Pin,14 ga.,7
